In [1]:
# Python core
from typing import Optional, Callable, TypedDict, Union, Iterable, Tuple, NamedTuple, List
from dataclasses import dataclass
import datetime
from itertools import product
from concurrent import futures

# Scientific python
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# PyTorch
# import torch
# from torch import nn
# import torch.nn.functional as F
# from torchvision import transforms
# import pytorch_lightning as pl

In [2]:
import bokeh
print('bokeh version: ', bokeh.__version__)

bokeh version:  3.2.1


In [13]:
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

...shutdown client...


In [4]:
from distributed import Client
from dask_jobqueue import PBSCluster

cluster = PBSCluster(account='NAML0001',walltime='12:00:00',cores=1, memory='100GB',shared_temp_directory='/glade/scratch/wchapman/tmp',queue='casper',interface='ext')
cluster.scale(jobs=40)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.63:36725,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Load the Zarr file

In [6]:
%%time
ZARR = '/glade/derecho/scratch/wchapman/STAGING/All_2010_staged.zarr'

plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['image.interpolation'] = 'none'

def get_forward_data(filename: str=ZARR) -> xr.DataArray:
    """Lazily opens the Zarr store on gladefilesystem.
    """
    dataset = xr.open_zarr(filename, consolidated=True)
    return dataset

forcing_data = get_forward_data().unify_chunks()

CPU times: user 2.13 s, sys: 94.6 ms, total: 2.22 s
Wall time: 2.31 s


## Take a Variety of Means/STDs and Save them
- you need a massive amount of compute to do this; try it in the morning before every other NCAR employee wakes up 

In [11]:
%%time
Mean_latlonlev = forcing_data.mean(['time','latitude','longitude','level']).persist()
print('...moving on to load...')
Mean_latlonlev = Mean_latlonlev.load()
print('...mean done...')
print('...saving...')
Mean_latlonlev.to_netcdf('/glade/derecho/scratch/wchapman/STAGING/All_2010_staged.mean.nc')


/glade/work/wchapman/miniconda3.1/envs/MLWPS/lib/python3.11/site-packages/distributed/client.py:3125: UserWarning: Sending large graph of size 50.19 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


...moving on to load...


...mean done...
...saving...
CPU times: user 10min 54s, sys: 20.4 s, total: 11min 14s
Wall time: 29min 39s


In [12]:
STD_latlonlev = forcing_data.std(['time','latitude','longitude','level']).persist()
# print('...moving on to load...')
STD_latlonlev = STD_latlonlev.load()
print('...saving...')
STD_latlonlev.to_netcdf('/glade/derecho/scratch/wchapman/STAGING/All_2010_staged.std.nc')

/glade/work/wchapman/miniconda3.1/envs/MLWPS/lib/python3.11/site-packages/distributed/client.py:3125: UserWarning: Sending large graph of size 50.54 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


...moving on to load...


...saving...
